# Librerías

In [ ]:
import sys
import os
sys.path.append(".")
import re
import copy
import pandas as pd
import numpy as np
import pandasql as ps
from datetime import timedelta
from matplotlib.ticker import FuncFormatter
import matplotlib.pyplot as plt
import seaborn as sns

color_pal = sns.color_palette()
plt.style.use('fivethirtyeight')
pd.set_option('display.max_columns', None)
def thousands_formatter_func(x, pos):
    return f'{int(x / 1e3)}K'
thousand_formatter = FuncFormatter(thousands_formatter_func)
def decimal_percentage_formatter_func(x, pos):
    return f'{int(x * 100)}%'
decimal_percentage_formatter = FuncFormatter(decimal_percentage_formatter_func)
def percentage_formatter_func(x, pos):
    return f'{int(x)}%'
percentage_formatter = FuncFormatter(percentage_formatter_func)

# Lectura archivos

In [ ]:
df_bombs = pd.read_csv('data/processed_csv/df_bombs.csv', delimiter=',', encoding='latin-1', index_col=0)
df_of = pd.read_csv('data/processed_csv/df_of.csv', delimiter=',', encoding='latin-1')
df_operators = pd.read_csv('data/processed_csv/df_operators.csv', delimiter=',', encoding='latin-1')
df_operators_participation = pd.read_csv('data/processed_csv/df_operators_participation.csv', delimiter=',', encoding='latin-1')

df_bombs['start_date'] = pd.to_datetime(df_bombs['start_date'], format='%Y-%m-%d %H:%M:%S.%f')
df_bombs['end_date'] = pd.to_datetime(df_bombs['end_date'], format='%Y-%m-%d %H:%M:%S.%f')
df_of['start_date'] = pd.to_datetime(df_of['start_date'], format='%Y-%m-%d %H:%M:%S.%f')
df_of['end_date'] = pd.to_datetime(df_of['end_date'], format='%Y-%m-%d %H:%M:%S.%f')



In [ ]:
df_of.line = df_of.line.replace({"LÃ\x8dNEA 2": "LINEA 2", "LINEA_2": "LINEA 2", "LINEA_4": "LINEA 4", "LÃ\x8dNEA 1": "LINEA 1", "LINEA_1": "LINEA 1", "LINEA_6": "LINEA 6",
                    "LINEA_3": "LINEA 3", "LÃ\x8dNEA KIVU": "LINEA KIVU", "PREFILTRO L-1": "PREFILTRO_L-1", "PREFILTRO L-6": "PREFILTRO_L-6",
                        "LINEA_7": "LINEA 7", "LINEA 8 IML": "LINEA_8_IML"})
df_operators_participation.line = df_operators_participation.line.replace({"LÃ\x8dNEA 2": "LINEA 2", "LINEA_2": "LINEA 2", "LINEA_4": "LINEA 4", "LÃ\x8dNEA 1": "LINEA 1", "LINEA_1": "LINEA 1", "LINEA_6": "LINEA 6",
                    "LINEA_3": "LINEA 3", "LÃ\x8dNEA KIVU": "LINEA KIVU", "PREFILTRO L-1": "PREFILTRO_L-1", "PREFILTRO L-6": "PREFILTRO_L-6",
                        "LINEA_7": "LINEA 7", "LINEA 8 IML": "LINEA_8_IML"})

df_of.line.unique()

In [ ]:
def remove_special_chars(text):
    # Agrega los caracteres especiales que deseas eliminar, incluyendo los tildes
    special_chars = r"[^\w\sáéíóúÁÉÍÓÚñÑÃ]"
    text_without_special_chars = re.sub(special_chars, '', text)
    # Agrega aquí cualquier otro reemplazo adicional que desees realizar
    return text_without_special_chars

In [ ]:
df_of['line'].apply(remove_special_chars)

In [ ]:
df_of["weekday"] = df_of["start_date"].dt.weekday.astype("category")
df_of["turn"] = df_of["start_date"].apply(lambda x: 'AM' if x.hour < 14 else 'PM')
df_of["month"] = df_of["start_date"].dt.month.astype("category")
df_of["year"] = df_of["start_date"].dt.year.astype("category")


In [ ]:
df_operators_participation.head()

# Cargar modelo

In [ ]:
import pickle

# Cargar el modelo desde el archivo
with open('classifier_model.pickle', 'rb') as file:
    classifier_model = pickle.load(file)

In [ ]:
classifier_model.feature_names_in_

# Algoritmo

In [ ]:
import pandas as pd
import unicodedata

def remove_accents(text):
    return ''.join(c for c in unicodedata.normalize('NFD', text) if unicodedata.category(c) != 'Mn')

def get_modified_dataframe():
    # Ruta completa del archivo CSV
    ruta_archivo = 'data/processed_csv/df_operators_participation.csv'

    # Leer el archivo CSV en un DataFrame
    df_operators_participation = pd.read_csv(ruta_archivo)

    df_operators_participation['line'] = df_operators_participation['line'].apply(lambda x: remove_accents(x))

    # Eliminar los registros que contienen 'PREFILTRO' en la columna 'line'
    df_operators_participation = df_operators_participation[~df_operators_participation['line'].str.contains('PREFILTRO')]

    return df_operators_participation

def active_operators_by_lines(dataframe, available_operators, available_lines):
    lineas_operarios_disponibles = {}

    for index, row in dataframe.iterrows():
        operador = str(row['operator_id'])
        linea = row['line']

        if linea in available_lines and operador in available_operators:
            if linea in lineas_operarios_disponibles:
                if operador not in lineas_operarios_disponibles[linea]:
                    lineas_operarios_disponibles[linea].append(operador)
            else:
                lineas_operarios_disponibles[linea] = [operador]

    return lineas_operarios_disponibles


def active_lines_by_order(dataframe, available_lines, orders):
    def lineas_por_bomba(dataframe):
        bombas_lineas = {}

        for index, row in dataframe.iterrows():
            linea = row['line']
            bomba = row['bomb_type']

            if bomba in bombas_lineas:
                if linea not in bombas_lineas[bomba]:
                    bombas_lineas[bomba].append(linea)
            else:
                bombas_lineas[bomba] = [linea]

        return bombas_lineas
    
    df = dataframe.copy()
    df['line'] = df['line'].apply(lambda x: remove_accents(x))

    bombas_lineas = lineas_por_bomba(df)

    active_lines = {}

    for order in orders:
        order_id = order['id']
        bomba = order['bomb_type']
        valid_lines = [line for line in available_lines if line in bombas_lineas.get(bomba, [])]

        if valid_lines:
            active_lines[order_id] = valid_lines

    return active_lines

available_operators = ['37', '5004', '5033', '8164', '8830', '8833', '8860', '8894', '9104', '9142', '9254', '9279', '9280']
available_lines = ['LINEA 3', 'LINEA 2', 'LINEA KIVU', 'LINEA 4', 'LINEA 7', 'LINEA 1', 'LINEA 6' ]

ORDERS = [
    {'id': 5365043,
    'bomb_type': '73676',
    'good_qty': 79.0,
    'theorical_time': 9.35},
    {'id': 5366240,
    'bomb_type': '01205-0810',
    'good_qty': 17.0,
    'theorical_time': 80.0},
    {'id': 5371350,
    'bomb_type': '65557H',
    'good_qty': 30.0,
    'theorical_time': 6.87},
    {'id': 5372841,
    'bomb_type': '66047-0890',
    'good_qty': 6.0,
    'theorical_time': 66.0},
    {'id': 5374961,
    'bomb_type': '25464',
    'good_qty': 60.0,
    'theorical_time': 7.1},
    {'id': 5375947,
    'bomb_type': '08003-0810',
    'good_qty': 9.0,
    'theorical_time': 21.54},
    {'id': 5376189,
    'bomb_type': '6862',
    'good_qty': 66.0,
    'theorical_time': 8.5}]

df_operators_participation = get_modified_dataframe()

active_operators_by_line = active_operators_by_lines(df_operators_participation, available_operators, available_lines)

active_lines_by_order = active_lines_by_order(df_operators_participation, available_lines, ORDERS)

In [ ]:
active_operators_by_line

In [ ]:
active_lines_by_order

In [ ]:
df_bombs_constraint = pd.read_excel('data/to_load/constraint/bombs_constraint.xlsx')
df_bombs_constraint

In [ ]:
df_bombs_constraint['Linea'] = df_bombs_constraint['Linea'].apply(lambda x: [i.strip() for i in x.split(',')])

diccionario = df_bombs_constraint.set_index('Bomb_type')['Linea'].to_dict()

print(diccionario)

In [ ]:
active_lines_by_order = {}

for order in ORDERS:
    bomb_type = str(order['bomb_type'])  # asegurarse de que bomb_type sea string
    order_id = order['id']
    if bomb_type in diccionario:
        active_lines_by_order[order_id] = diccionario[bomb_type]
    else:
        active_lines_by_order[order_id] = []  # si el bomb_type no está en el diccionario, añadir una lista vacía

print(active_lines_by_order)

In [ ]:
df_bombs_constraint.to_json()

In [ ]:
df_operators_constraint = pd.read_excel('data/to_load/constraint/operators_constraint.xlsx')
df_operators_constraint

In [ ]:
df_operators_constraint['Lineas'] = df_operators_constraint['Lineas'].apply(lambda x: [i.strip() for i in x.split(',')])

diccionario = df_operators_constraint.set_index('Operario')['Lineas'].to_dict()

print(diccionario)

In [ ]:
lines_by_operator = {}

for _, row in df_operators_constraint.iterrows():
    for line in row['Lineas']:
        if line in lines_by_operator:
            lines_by_operator[line].append(row['Operario'])
        else:
            lines_by_operator[line] = [row['Operario']]

lines_by_operator

In [ ]:
# LINE = 'LINEA_3', 'LINEA_2', 'LINEA_KIVU', 'LINEA 4', 'LINEA_7', 'LINEA_1', 'LINEA_6' 
# CUMPLE = '0', '0', '0', '0', '0', '1', '1' 
example_orders = [5365043, 5371350, 5372841, 5374961, 5376189, 5366240, 5375947]

In [ ]:
df_of[df_of['order'].isin(example_orders)]

In [ ]:
active_lines_by_order

In [ ]:
# ORDERS = [
#     {'id': 5160396, 'good_qty': 100, 'theorical_time': 2.5, 'bomb_type': '01224-0890'},
#     {'id': 5169247, 'good_qty': 200, 'theorical_time': 3.0, 'bomb_type': '1210'},
#     {'id': 5171973, 'good_qty': 150, 'theorical_time': 2.0, 'bomb_type': '1197'},
# ]

# ORDERS_DICT = {
#     5160396: {'good_qty': 100, 'theorical_time': 2.5, 'registers_qty': 30, 'operators_distinct_qty': 2, 'days_accumulated_experience': 102, 'OFs_accumulated_experience': 50},
#     5169247: {'good_qty': 200, 'theorical_time': 3.0, 'registers_qty': 10, 'operators_distinct_qty': 3, 'days_accumulated_experience': 102, 'OFs_accumulated_experience': 50},
#     5171973: {'good_qty': 150, 'theorical_time': 2.0, 'registers_qty': 50, 'operators_distinct_qty': 4, 'days_accumulated_experience': 102, 'OFs_accumulated_experience': 50},
# }


# available_operators = ['37', '5004', '5033', '8164', '8830', '8833', '8860', '8894', '9104', '9142', '9254', '9279', '9280']
# available_lines = ['LINEA 3', 'LINEA 2', 'LINEA KIVU', 'LINEA 4', 'LINEA 7', 'LINEA 1', 'LINEA 6' ]

# active_lines_by_order = {
#     5160396: ['LINEA_1', 'LINEA_2'],
#     5169247: ['LINEA_1', 'LINEA_2', 'LINEA_3'],
#     5171973: ['LINEA_2', 'LINEA_3']
# }

# active_operators_by_line = {
#     'LINEA_1': [9395, 9391, 9378],
#     'LINEA_2': [9380, 9378, 9352],
#     'LINEA_3': [9349, 9391]
# }

In [ ]:
from deap import creator, base, tools, algorithms
import random
import itertools

In [ ]:
def format_df_to_classifier_model(order, operators, line, available_operators, available_lines):
    # DataFrame de ceros con las columnas que necesitamos
    return True
    columns = ['good_qty', 'theorical_time', 'registers_qty', 'operators_distinct_qty'] + ['operator_' + str(i) for i in available_operators] + ['line_' + line for line in available_lines] + ['days_accumulated_experience', 'OFs_accumulated_experience']
    order_df = pd.DataFrame(np.zeros((1, len(columns))), columns=columns)
    # Actualiza las columnas relevantes
    order_df['good_qty'] = order['good_qty']
    order_df['theorical_time'] = order['theorical_time']
    order_df['registers_qty'] = order['registers_qty']
    order_df['operators_distinct_qty'] = len(operators)
    order_df['days_accumulated_experience'] = order['days_accumulated_experience']
    order_df['OFs_accumulated_experience'] = order['OFs_accumulated_experience']
    for operator in operators:
        order_df['operator_' + str(operator)] = 1

    order_df['line_' + line] = 1
    return order_df

In [ ]:
unique_individuals = set()
evaluations_count = 0
def main():
    global evaluations_count, unique_individuals
    unique_individuals = set()
    evaluations_count = 0
    def individual_to_str(individual):
        return str(sorted((assignment['order'], tuple(sorted(assignment['operators'])), assignment['line']) for assignment in individual))

    def evaluate(individual):
        global evaluations_count, unique_individuals
        evaluations_count += 1
        # Agrega el individuo al conjunto de individuos únicos
        unique_individuals.add(individual_to_str(individual))

        success_count = 0
        total_probability = 0.0
        for assignment in individual:

            order_id = assignment['order']
            operators = assignment['operators']
            line = assignment['line']

            # Encuentra la orden correspondiente en la lista de órdenes
            order = next(order for order in ORDERS if order['id'] == order_id)
            
            order_df = pd.DataFrame({
                'good_qty': [order['good_qty']],
                'theorical_time': [order['theorical_time']],
                'operators': [operators],
                'line': [line],
            })
            format_df_to_classifier_model(order, operators, line, available_operators, available_lines)
            # prediction = classifier_model.predict(order_df)[0]
            prediction_proba = random.random()
            prediction = 1 if prediction_proba > 0.5 else 0

            if prediction == 1:  # Si la OF es exitosa
                success_count += 1
            total_probability += prediction_proba
        
        fitness = success_count + (total_probability / 10000) 
        return fitness,

    operator_combos = []
    max_operators_per_order = 4
    for r in range(1, max_operators_per_order + 1):
        operator_combos.extend(itertools.combinations(available_operators, r))
    
    # Definimos los tipos para el problema de optimización
    creator.create("FitnessMax", base.Fitness, weights=(1.0,))  # Maximizamos la función de fitness
    creator.create("Individual", list, fitness=creator.FitnessMax)

    toolbox = base.Toolbox()

    

    def initIndividual():
        attempts = 0
        max_attempts = 10000  # ajusta este número según sea necesario
        while attempts < max_attempts:
            individual = []
            available_operators_for_this_individual = available_operators.copy()  # Crear una copia de la lista de operadores disponibles
            available_lines_for_this_individual = available_lines.copy()  # Crear una copia de la lista de líneas disponibles
            for order in ORDERS:
                # Elegir una línea que aún no se haya asignado
                posible_lines_for_this_order = [line for line in available_lines_for_this_individual if line in active_lines_by_order[order['id']]]
                if not posible_lines_for_this_order:  # Si no hay más líneas disponibles, no podemos generar un individuo válido
                    break
                line = random.choice(posible_lines_for_this_order)
                available_lines_for_this_individual.remove(line)  # La línea ya no está disponible para las siguientes órdenes

                # Elegir un conjunto de operadores que aún no se hayan asignado
                posible_operators_for_this_line = [operator for operator in available_operators_for_this_individual if operator in active_operators_by_line[line]]
                valid_operator_combos = [combo for combo in operator_combos if set(combo).issubset(posible_operators_for_this_line)]
                if not valid_operator_combos:  # Si no hay más operadores disponibles, no podemos generar un individuo válido
                    break
                operators = random.choice(valid_operator_combos)
                for operator in operators:
                    available_operators_for_this_individual.remove(operator)  # El operador ya no está disponible para las siguientes órdenes

                individual.append({'order': order['id'], 'operators': operators, 'line': line})

            if len(individual) == len(ORDERS):  # si el individuo es válido (tiene todas las órdenes)
                print(f'[INIT INDIVIDUAL]: {attempts}')
                print(individual)
                return creator.Individual(individual)

            attempts += 1

        # Si hemos llegado a este punto, es porque todos los intentos de generar un individuo válido han fallado.
        raise Exception(f"No se pudo generar un individuo válido después de {max_attempts} intentos.")



    toolbox.register("individual", initIndividual)
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)

    # Definimos los operadores genéticos
    toolbox.register("evaluate", evaluate)


    # Definimos la función de crossovr
    def custom_crossover(ind1, ind2):
        # Crea copias de los individuos para no modificar los originales
        modified = False
        new_ind1 = copy.deepcopy(ind1)
        new_ind2 = copy.deepcopy(ind2)
        
        for order in ORDERS:
            order_id = order['id']

            # Encuentra las asignaciones para esta orden en ambos individuos
            assignment1 = next(assignment for assignment in new_ind1 if assignment['order'] == order_id)
            assignment2 = next(assignment for assignment in new_ind2 if assignment['order'] == order_id)
            

            # No se requiere validación de order y line porque se garantiza que los individuos son válidos
            # Solamente se valida el intercambio de operadores
            if (set(assignment2['operators']).issubset(active_operators_by_line[assignment1['line']])
                    and set(assignment1['operators']).issubset(active_operators_by_line[assignment2['line']])):
                # Intercambia las asignaciones de operadores y líneas
                assignment1['operators'], assignment2['operators'] = assignment2['operators'], assignment1['operators']
                assignment1['line'], assignment2['line'] = assignment2['line'], assignment1['line']
                modified = True
        
        if not modified:
            print('[ERROR-CROSSOVER NOT MODIFIED]:')
            print(new_ind1)
            print(new_ind2)
            return ind1, ind2

        # Verifica si las nuevas asignaciones violan las restricciones
        violation = False
        for new_ind in [new_ind1, new_ind2]:
            for i in range(len(new_ind1)):
                for j, assignment in enumerate(new_ind):
                    if j != i:
                        # Verifica si la nueva línea se repite en otras asignaciones
                        if new_ind[i]['line'] == assignment['line']:
                            violation = True
                            break

                        # Verifica si los nuevos operadores se repiten en otras asignaciones
                        if any(op in new_ind[i]['operators'] for op in assignment['operators']):
                            violation = True
                            break
        if violation:
            print('[ERROR-CROSSOVER VIOLATION]:')
            print(new_ind1)
            print(new_ind2)
            return ind1, ind2
        
        print('[CROSSOVER GENERATED]')
        return new_ind1, new_ind2

    toolbox.register("mate", custom_crossover)

    def custom_mutation(individual, indpb):
        attempts = 0
        for i in range(len(individual)):
            if random.random() < indpb:
                attempts = 0
                max_attempts = 10000  # ajusta este número según sea necesario
                while attempts < max_attempts:
                    # Selecciona una nueva línea dentro de las posibles:
                    new_line = random.choice(active_lines_by_order[individual[i]['order']])

                    # Verifica si la nueva asignación viola las restricciones
                    violation = False
                    posible_operators_for_this_line = [operator for operator in active_operators_by_line[new_line]]
                    valid_operator_combos = [combo for combo in operator_combos if set(combo).issubset(posible_operators_for_this_line)]
                    new_operators = random.choice(valid_operator_combos)
                    for j, assignment in enumerate(individual):
                        if j != i:
                            # Verifica si la nueva línea se repite en otras asignaciones
                            if new_line == assignment['line']:
                                violation = True
                                break

                            # Verifica si los nuevos operadores se repiten en otras asignaciones
                            if any(op in new_operators for op in assignment['operators']):
                                violation = True
                                break

                    if not violation:
                        # Si la asignación es válida, actualiza la asignación en el individuo
                        individual[i]['line'] = new_line
                        individual[i]['operators'] = tuple(new_operators)  # Convierte la lista en una tupla
                        break

                    attempts += 1
        print(f'[MUTATION GENERATED]: {attempts or 0} intentos')
        return individual,

    toolbox.register("mutate", custom_mutation, indpb=0.4)
    toolbox.register("select", tools.selTournament, tournsize=3)

    # Inicializamos la población y ejecutamos el algoritmo genético
    population = toolbox.population(n=50)
    result = algorithms.eaSimple(population, toolbox, cxpb=0.5, mutpb=0.2, ngen=40, verbose=False)
    best_individual = tools.selBest(result[0], 3)[0]
    best_fitness = best_individual.fitness.values

    print('···· RESULTADOS ····')
    print(f'Se han realizado {evaluations_count} evaluaciones.')
    # print(f'Se han realizado {len(unique_individuals)} evaluaciones únicas.')
    print(f'El mejor individuo es: {best_individual}')
    def selectUnique(individuals, k):
        unique_individuals = []
        for ind in individuals:
            if not any(ind2 == ind for ind2 in unique_individuals):
                unique_individuals.append(ind)
            if len(unique_individuals) == k:
                break
        return unique_individuals

    unique_individuals = selectUnique(tools.selNSGA2(result[0], len(result[0])), 10)
    best_fitnesses = [ind.fitness.values for ind in unique_individuals]

    for i in range(len(unique_individuals)):
        print('########################')
        print(f'\nMejor individuo {i+1}:')
        print(f'Valor de fitness: {best_fitnesses[i]}')
        for assignment in unique_individuals[i]:
            order_id = assignment['order']
            operators = assignment['operators']
            line = assignment['line']
            print(f'  Orden: {order_id}, Operadores: {operators}, Línea: {line}')


In [ ]:
main()

Violation Code

In [ ]:
example = [{'order': 5160396, 'operators': (1, 2, 9), 'line': 'LINEA_3'}, {'order': 5169247, 'operators': (3, 4), 'line': 'LINEA_2'}, {'order': 5171973, 'operators': (3,), 'line': 'LINEA_4'}]

In [ ]:
for i in range(len(example)):
    violation = False
    for j, assignment in enumerate(example):
        if j != i:
            # Verifica si la nueva línea se repite en otras asignaciones
            if example[i]['line'] == assignment['line']:
                violation = True
                break

            # Verifica si los nuevos operadores se repiten en otras asignaciones
            if any(op in example[i]['operators'] for op in assignment['operators']):
                violation = True
                break
print(violation)

# Pruebas

In [ ]:
orders = ['5376213', '5372454', '5376066', '5373384', '5379842', '5382059', '5374960']
orders = [5376213, 5372454, 5376066, 5373384, 5379842, 5382059, 5374960]

In [ ]:
df_of.line.unique()

In [ ]:
lines = ['LINEA_2', 'LINEA 4', 'LINEA 6', 'LINEA_1', 'LINEA_3', 'LINEA_KIVU', 'LINEA_7']

In [ ]:
df_bombs.reset_index(inplace=True)
df_bombs[df_bombs['order'].isin(orders)]

In [ ]:
df_of[(df_of['year'] == 2023) & (df_of['line'] == 'LINEA 6')]

In [ ]:
# LINE = 'LINEA_3', 'LINEA_2', 'LINEA_KIVU', 'LINEA 4', 'LINEA_7', 'LINEA_1', 'LINEA_6' 
# CUMPLE = '0', '0', '0', '0', '0', '1', '1' 
orders = [5365043, 5371350, 5372841, 5374961, 5376189, 5366240, 5375947]

In [ ]:
df_of[df_of['order'].isin(orders)]

In [ ]:
df_bombs[df_bombs['order'].isin(orders)]